# <center>Tutorial on Q-Leraning</center>

In [ ]:
import copy
import gym
import os

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import utils
from Env import Env
from GridWord import GridWord
from LearningRate import LearningRate
from Q import Q
from PolicyEpsilonGreedy import PolicyEpsilonGreedy

from math import radians
import time

# Theoritical introduction

## Markovian Decision Porcess

Before tackling a reinforcement learning problem let's define a modelisation for the agents. The <b>Markovian Decision Process (MDP)</b> is defined as follows:
* A <b>state</b> space $\mathcal{X}$.
* An <b>action</b> space $\mathcal{A}$.
* A <b>transition</b> transition probability distribution $p(x'|x,a)$. This corresponds to the dynamics of the problem. "What is the space when this action is taken while being in this state".
* A <b>reward</b> function $r(x,a)$. "What do we get from performing this action while being in this state".

This modelisation is Markovian since it follows the <b>Markov property</b>:

\begin{equation}
    \mathbb{P}(X_t = x | X_0, \dots, X_{t-1}) = \mathbb{P}(X_t = x | X_{t-1})
\end{equation}

In other words, the evolution of the agent only depends on the state he is in, not on the past. This will be usefull to train the agent.

## The Value function and the Bellman equation

Let's define the <b>Value function</b> $V$ as the total reward we can expect when we are in a certain state. This depends on the <b>policy</b> we choose. The policy $\pi(a|x)$ is defined by the operator, and defines what action is to be taken depending on the state. Then the value function is defined by:

\begin{equation}
    V^\pi(x) = \mathbb{E}\Big[\sum_{t=0}^T \gamma^t r(x_t, a_t) \Big| x_0, \pi \Big]
\end{equation}
Where $\gamma$ is the discount over time and $T$ is the random final time step.

Let's denote by $\pi^*$ the optimal policy, that is to say the policy that maximizes the value function at each state, and by $V^*$ the resulting value function.

If the policy is stationary, meaning that it doesn't change with time, wich is the case for the optimal policy, then the value function follows the <b>Bellman equation</b>:

\begin{equation}
    V^*(x) = r(x,\pi^*(x)) + \gamma \sum_{x' \in \mathcal{X}} p(x'|x,\pi^*(x)) V^*(x')
\end{equation}
Where $\pi^*(x)$ is the action returned by the optimal policy.

## Q-Learning

Instead of trying to learn the value fonction $V(x)$ for all states, Q-learning consists of learning the $Q$ function that associates each couple of state and action with the expected final reward.
The associated value function of a $Q$ function under the policy $\pi$ is:

\begin{equation}
    V^{\pi}(x) = \sum_{a \in \mathcal{A}} \pi(a|x) Q(x, a)
\end{equation}

If we define the optimal policy as the greedy policy $\pi^*(x) = \arg\max_a Q^*(x, a)$. Then the optimal $Q$ function follows the Bellman equation. We can use this to approximate $Q^*$ with the <b>Q-learning</b> algorithm:

<b>Initialize</b> $Q_0$ randomly  
<b>For</b> $e = 1, \dots,n$:  
> <b>Intiate</b> $t = 0, x_0$  
> <b>While</b> $x_t$ <b>not</b> terminal:  
>> <b>Choose</b> $a_t$ according to a suitable <b>exploration policy</b>  
>> <b>Observe</b> $r_t = r(x_t, a_t)$, $x_{t+1}$  
>> <b>Temporal difference</b> $\delta_{t} = r_t$ <b>If</b> $x_t$ is terminal <b>Else</b> $r_t + \gamma \max_{a}Q(x_{t+1}, a) - Q(x_t, a)$  
>> <b>Update</b> $Q(x_t, a_t) =  Q(x_t, a_t) + \alpha(x_t, a_t) \delta_t$  

If the learning rate $\alpha_t$ satisfies the <b>Robbins-Monro</b> conditions $\sum_{t=0}^{\infty} \alpha_t = \infty$ and $\sum_{t=0}^{\infty} \alpha_t^2 < \infty$ for every state action couple, and every state is visited infinitely often then the convergence toward the optimal $Q$ function is guaranted (almost surrely).

# Presentation of the provided Classes

### The first environement we are going to use is a maze:
```python
GridWord
# methods
    __init__(path) # Only needs a path to the grid file that describes the maze
    reset() # resets the environement to it's original state
        return state
    step(action) # Performs the requiered action
                 # action has too be in ["up", "right", "down", "left"]
        return state, reward, done # Please note thate the reward is allways -1 in this environement
    render_path() # Plots the maze and the path taken by the agent up to now
    render_V() # Plots the value function associated with the Q function of the environement
    render_path_and_V() # Both.
```

Note that ```GridWord``` also has an instance of the next ```Q``` class initialized.
### The Q function:
```python
Q
# methods
    __init__(actions_size, # Can either be a list of the actions or the size of the action space
             discrete=True, # If the STATE space is discrete or not
             state_shape=[], # If the state space is discrete provide its shape
             segmentation=[], # If it is continuous provide a discritization grid
             init_range=[0,1]) # Initialization range for Q.
    __call__(state, action)
        return Q_value
    get_V(state) # Returns the value funciton for this state
        return V_value
```

### The $\epsilon$-Greedy Policy

Earlier we talked about a <b>"suitable exploration ppolicy"</b>. 
The $\epsilon$-greedy policy chooses an action as follows:
* With probability $\epsilon$ returns $\arg\max_a Q(x_t, a)$
* With probability $1-\epsilon$ uniformly chooses <u>another</u> state

```python
PolicyEpsilonGreedy
# methods
    __init__(Q, action_space, # An instance of the previous class and the action space
             epsilon, decay, # The initial epsilon and his decay rate
             lower, decay_every) # A lower bound for epsilon and the rate at which decay is performed
    __call__(state, be_greedy=False) # If be_greedy is True ignores epsilon and returns the best action
        return action                # usefull at test time
```

### Learning-Rate

In Q-learning the learning rate depends on the (state, action) couple. Moreover the  learning rate has to satisfy the Robbins-Monro conditions. Here we are going to use the following learning rate:
* $\alpha(x, a) = \alpha_0 / (1+t(x,a)\beta)$

Where $t(x, a)$ is the number of time this state action couple has been visited.

```python
LearningRate
# methods
    __init__(lr0, decay, min_lr, ...) # The initial learning rate, beta and the minimum lr
                                      # The other arguments are the same than those used to define Q
    __call__(state, action)
        return alpha(x,a)
```

# A few theoritical questions

* <b> Why do you think it is important to use the $\epsilon$-greedy policy and not just the greedy policy ?</b>
* <b> Why is the learning rate independant for every (state, action) couple ?</b>
* <b> Why do you think the Robbins-Monro conditions are important ?   

# A toy exemple

Let's consider a simple exemple to illustrate the principles introduces above.

In [ ]:
gridword = GridWord("./GridWordResults/grid.txt")

for action in ["up", "right", "right", "up", "up", "left", "up"]:
    gridword.step(action)

gridword.render_path_and_V()

### Please define this problem as a MDP:

#### Your answer <b>...</b>

## Now we are ready to implement the Q-learning alogrithm

In the next cell you are asked to implement a function that takes as input a ```Q``` instance, an ```env``` and learns the Q function. You only need the ```PolicyEpsilonGreedy``` and ```LearningRate``` classes for that. Please also return the rewards history so that we can plot the learning curve.

In [ ]:
def Q_learning(Q_, env, action_space, gamma, print_every_episode, n_episodes, epsilon, epsilon_decay, epsilon_min,
              epsilon_decay_every, lr0, lr_decay, min_lr):
    policy = PolicyEpsilonGreedy(Q_, action_space, epsilon, epsilon_decay, epsilon_min,
                                  epsilon_decay_every)
    lr = LearningRate(lr0=lr0, decay=lr_decay, min_lr=min_lr, discrete=Q_.discrete,
                      actions_size=action_space, state_shape=gridword.grid.shape)
    
    '''
            YOUR CODE HERE
    ''' 
    return Q, policy, rewards

In [ ]:
# Initializing the environement
gridword = GridWord("./GridWordResults/grid.txt")
action_space = ["up", "right", "down", "left"]

# You can play with all the parameters
# Try to find the best ones (those provided are random)
_, policy, rewards = Q_learning(Q_=gridword.Q, env=gridword, action_space=action_space,
                                print_every_episode=20, n_episodes=500, gamma=0.6, 
                                epsilon=0.01, epsilon_decay=0.4, epsilon_min=0.001, epsilon_decay_every=10, 
                                lr0=4, lr_decay=0.0001, min_lr=0.001)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(utils.rolling_average(rewards, 10))
plt.title("Learning curve")
plt.show()

In [ ]:
state = gridword.reset()
done = False
while not done:
    action = policy(state, be_greedy=True)
    state, _, done = gridword.step(action)

gridword.render_path_and_V()

# A more complex problem

We are going to use <a href="https://gym.openai.com">OpenAI Gym</a>. This is a toolkit for developing and comparing reinforcement learning algorithms. It supports a lot of environement and it is easy to play with.

Let's start with a simple environement <b>CartPole</b>. The goal is to balance a stick.

In [ ]:
# This cells shows how to set up a gym environement and run it with a random policy
# We also save the run in the directory "./gym-result/test" so that we can watch the video later.
env = Env('CartPole-v1', "./gym-results/CartPole/test")
_ = env.run(100, policy="random")
# Now lets display the run
env.display_run()

As we can see above, with a random policy the agent quickly looses all it's life fastly reaching a game over. <b>With thus need to implement some clever policy</b>.

### Please define this problem as a MDP:

#### Your answer <b>...</b>

In [ ]:
# Let's define a more clever policy by going in the same direction than our speed.
def policy_smart(state):
    if state[3] > 0:
        return 1
    
    elif state[3] <= 0:
        return 0

env = Env('CartPole-v1', "./gym-results/CartPole/smart")
rewards = env.run(500, policy=policy_smart)
env.display_run()

In [ ]:
# To understand every components lets run some random experiments and visualize the states.

env = gym.make('CartPole-v0')
states = []
actions = []
for n_samples in range(1000):
    env.reset()
    for _ in range(100):
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        actions.append(action)
        states.append(state)
env.close()

component_names = ["x_pos", "x_vel", "theta", "theta_vel"]
fig = plt.figure(figsize=(16,12))
for _ in range(4):
    plt.subplot(2,4,_+1)
    plt.plot([state[_] for state in states]) 
    plt.title("State component %i, %s" % (_, component_names[_]))
plt.show()

In [ ]:
# Initializing the environement
env = gym.make('CartPole-v0')
action_space = [0,1]

# Try to find a suitable segmentation
# If the list is empty it means you want to map all possible value of the states dimension to the same Q_values
segmentation = [[],
                [],
                [-1,1]
                [-1,1]]

Q_ = Q(init_range=[0,0],
       discrete=False, segmentation=segmentation, actions_size=len(action_space)) 

_, policy, rewards = Q_learning(Q_=Q_, env=env, action_space=len(action_space),
                                print_every_episode=20, n_episodes=500, gamma=0.6, 
                                epsilon=0.01, epsilon_decay=0.4, epsilon_min=0.001, epsilon_decay_every=10, 
                                lr0=4, lr_decay=0.0001, min_lr=0.001)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(utils.rolling_average(rewards, 50))
plt.title("Learning curve")
plt.show()

In [ ]:
env = Env('CartPole-v0', "./gym-results/CartPole/RL")
_ = env.run(200, policy)
env.display_run()

# Now lets land a space craft !

In [ ]:
env = Env('LunarLander-v2', "./gym-results/LunarLander/random")
_ = env.run(100, policy="random")
env.display_run()

In [ ]:
env = gym.make('LunarLander-v2')
states = []
actions = []
for n_samples in range(100):
    env.reset()
    for _ in range(100):
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        actions.append(action)
        states.append(state)
env.close()

component_names = ["x_pos", "y_pos", "x_vel", "y_vel", "angle", "angular_vel", "leg_1_sttus", "leg_2_status"]
fig = plt.figure(figsize=(16,12))
for _ in range(8):
    plt.subplot(2,4,_+1)
    plt.plot([state[_] for state in states]) 
    plt.title("State component %i, %s" % (_, component_names[_]))
plt.show()

In [ ]:
env = gym.make('LunarLander-v2')
action_space = [0,1,2,3]

segmentation = [[-1, 0, 1],
                [-0.5, 0, 1, 1.5],
                [-1, 0, 1],
                [-1,0],
                [-2,0,2],
                [-4, -2, 0, 2, 4],
                [],
                []]

Q_ = Q(init_range=[0,0],
       discrete=False, segmentation=segmentation, actions_size=len(action_space)) 

_, policy, rewards = Q_learning(Q_=Q_, env=env, action_space=len(action_space),
                                print_every_episode=50, n_episodes=1000, gamma=0.99,
                                epsilon=0.5, epsilon_decay=0.9, epsilon_min=0.1, epsilon_decay_every=50, 
                                lr0=1, lr_decay=0.1, min_lr=0.1)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(utils.rolling_average(rewards, 50))
plt.title("Learning curve")
plt.show()

In [ ]:
env = Env('LunarLander-v2', "./gym-results/LunarLander/smart")
_ = env.run(200, policy)
env.display_run()